In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from linkingtool import timeseries

In [28]:
CFP24=gpd.read_file('BC_CFP24.geojson')
# CFP24=pd.read_csv('BC_CFP24.csv')



In [51]:
CFP24.to_file('BC_CFP24.geojson', driver='GeoJSON')
CFP24.drop(columns='geometry', errors='ignore').to_csv('BC_CFP24.csv', index=False)

In [12]:
wind=pd.read_csv('../results/linking/resource_options_wind.csv')
solar=pd.read_csv('../results/linking/resource_options_solar.csv')

In [16]:
# CFP24.rename(columns={'capacity_MW': 'potential_capacity', 'gadm_region': 'Region'}, inplace=True)


In [49]:
import numpy as np

CFP24['capex'] = np.where(CFP24['resource_type'] == 'solar',  solar.loc[:,'capex'].mean(), wind.loc[:,'capex'].mean())
CFP24['Operational_life'] = np.where(CFP24['resource_type'] == 'solar',  solar.loc[:,'Operational_life'].mean(), wind.loc[:,'Operational_life'].mean())
CFP24['vom'] = np.where(CFP24['resource_type'] == 'solar', solar.loc[:,'vom'].mean(), wind.loc[:,'vom'].mean())
CFP24['fom'] = np.where(CFP24['resource_type'] == 'solar',  solar.loc[:,'fom'].mean(), wind.loc[:,'fom'].mean())

In [50]:
CFP24

,ERA5_cell,project_name,longitude,latitude,regional_system,expected_cod,poi,potential_capacity,annual_energy_GWh,Region,resource_type,capex,Operational_life,geometry,vom,fom
0,PeaceRiver_-121.0_56.0,Stewart Creek Wind Project,-120.980556,55.96222,Peace River,01-Dec-30,2L391,199.9,714.0,PeaceRiver,wind,1735.274516,20.0,POINT (-120.98056 55.96222),41.2222,0.0
1,PeaceRiver_-120.75_56.0,Taylor Wind Project,-120.852611,56.02683,Peace River,30-Sep-31,2L392,200.0,649.0,PeaceRiver,wind,1735.274516,20.0,POINT (-120.85261 56.02683),41.2222,0.0
2,Thompson-Nicola_-121.0_50.5,Highland Valley Wind Project,-121.047222,50.57778,Kelly Nicola,30-Sep-31,1L055,197.2,528.0,Thompson-Nicola,wind,1735.274516,20.0,POINT (-121.04722 50.57778),41.2222,0.0
3,Thompson-Nicola_-120.5_50.25,Mount Mabel Wind Project,-120.571389,50.30694,Kelly Nicola,31-Dec-30,1L243,142.8,370.0,Thompson-Nicola,wind,1735.274516,20.0,POINT (-120.57139 50.30694),41.2222,0.0
4,Thompson-Nicola_-120.75_50.5,ShTSaQU Solar Project,-120.842600,50.54810,Kelly Nicola,30-Sep-30,1L203,104.0,215.0,Thompson-Nicola,solar,1482.683280,20.0,POINT (-120.84260 50.54810),23.7656,0.0
5,Thompson-Nicola_-120.25_50.0,K2 Wind Project,-120.150833,49.88750,Kelly Nicola,01-Oct-31,1L244,159.6,575.0,Thompson-Nicola,wind,1735.274516,20.0,POINT (-120.15083 49.88750),41.2222,0.0
6,Fraser-FortGeorge_-122.25_53.5,Nilhts'i Ecoener Project (Formerly Hixon Wind ...,-122.333200,53.42240,Central Interior,30-Sep-31,2L096,140.4,346.0,Cariboo,wind,1735.274516,20.0,POINT (-122.33320 53.42240),41.2222,0.0
7,Comox-Strathcona_-126.0_50.25,Brewster Wind Project,-125.961722,50.25750,Vancouver Island,30-Sep-31,2L154,197.2,510.0,Capital,wind,1735.274516,20.0,POINT (-125.96172 50.25750),41.2222,0.0
8,Thompson-Nicola_-120.25_50.0,Boulder and Elkhart Wind Project,-120.236696,49.97332,Kelly Nicola,01-Oct-30,1L244,94.4,316.0,Thompson-Nicola,wind,1735.274516,20.0,POINT (-120.23670 49.97332),41.2222,0.0
9,Bulkley-Nechako_-124.75_54.0,Nithi Mountain Wind Project,-124.820639,53.91613,North Coast,01-Dec-30,GLN,199.9,607.0,Skeena-QueenCharlotte,wind,1735.274516,20.0,POINT (-124.82064 53.91613),41.2222,0.0


In [ ]:
from linkingtool.hdf5_handler import DataHandler
store=f"../data/store/resources_BC.h5"

timeseries.get_timeseries_for_project_points(store,
                                             CFP24, 
                                             'ROD_2024', 
                                             show=True)

In [108]:
rod=pd.read_excel('rod_202409.xlsm',sheet_name="Resource Options",skiprows=1)

In [109]:
# Convert to float to ensure proper type
rod['Latitude'] = pd.to_numeric(rod['Latitude'], errors='coerce')
rod['Longitude'] = pd.to_numeric(rod['Longitude'], errors='coerce')

# Drop rows with invalid coordinates
rod = rod.dropna(subset=['Latitude', 'Longitude'])

# Create geometry column with correct longitude, latitude order
rod['geometry'] = rod.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Convert to GeoDataFrame
rod_gdf = gpd.GeoDataFrame(rod, geometry='geometry', crs='EPSG:4326')
rod_gdf.to_pickle('ROD_processed.pkl')

In [110]:
# Create a mask for rows with 'Solar' or 'Onshore Wind' in 'Resource Type'
VREs_mask = ( (rod_gdf['Resource Type'] == 'Onshore Wind'))# |rod_gdf['Resource Type'] == 'Solar') 
# Apply the mask to filter the GeoDataFrame
filtered_rod_gdf = rod_gdf[VREs_mask]

# Display the filtered GeoDataFrame
filtered_rod_gdf=filtered_rod_gdf.head(50)
filtered_rod_gdf['Rank'] = range(1, len(filtered_rod_gdf) + 1)

In [111]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"../data/store/resources_BC.h5"
datahandler=DataHandler(store)
boundary=datahandler.from_store('boundary')

In [ ]:
clusters_wind.head(10).explore('LCOE')

In [116]:
cells=datahandler.from_store('cells')
cells=cells[cells['potential_capacity_wind']>7]
clusters_wind=datahandler.from_store('clusters/wind')#.head(5)
clusters_solar=datahandler.from_store('clusters/solar').head(2)

In [ ]:
import folium

# Initialize a Folium map centered at the mean coordinates of the data
map_center = [rod_gdf.geometry.y.mean(), filtered_rod_gdf.geometry.x.mean()]
m = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")  # Clean, modern map tiles

# Create FeatureGroups for each type of layer
boundary_group = folium.FeatureGroup(name="Boundary Layer")
wind_cells_group = folium.FeatureGroup(name="Wind Cells")
wind_clusters_group = folium.FeatureGroup(name="Wind Clusters")
solar_clusters_group = folium.FeatureGroup(name="Solar Clusters")
points_group = folium.FeatureGroup(name="Project Points")

# Add the boundary GeoDataFrame as a GeoJson layer
boundary_layer = folium.GeoJson(
    boundary,  # This is your boundary GeoDataFrame
    style_function=lambda x: {
        'fillColor': 'None',  # No fill color for the boundary
        'color': 'slategray',  # Border color for boundary regions
        'weight': .5,
        'ls':":",
        'fillOpacity': 0.2  # Transparency level
    }
)
boundary_group.add_child(boundary_layer)

# Add the MultiPolygon geometries (Wind Cells) directly to the map as GeoJson
for _, row in cells.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'violet',  # Color of the polygon
            'color': 'violet',  # Border color of the polygon
            'weight': .1,  # Border thickness
            'fillOpacity': 0.1  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{row['potential_capacity_wind']}<br>"
                               f"Region: {row['Region']}<br>"
                               f"LCOE ($/MWh): {row['LCOE_wind']}<br>"
                               f"Windspeed_GWA: {round(row['windspeed_gwa'], 2)} m/s")  # Show info on hover
    ).add_to(wind_cells_group)

# Add the MultiPolygon geometries (Wind Clusters) directly to the map as GeoJson
for _, row in clusters_wind.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'violet',  # Color of the polygon
            'color': 'violet',  # Border color of the polygon
            'weight': 0.5,  # Border thickness
            'fillOpacity': 0.5  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{round(row['potential_capacity']/1E3,2)} GW <br>"
                               f"Region: {row['Region']}<br>"
                               f"Rank: {row['Rank']}<br>"
                               f"LCOE ($/MWh): {round(row['LCOE'],2)}<br>"
                               )  # Show info on hover
    ).add_to(wind_clusters_group)

# Add the MultiPolygon geometries (Solar Clusters) directly to the map as GeoJson
for _, row in clusters_solar.iterrows():
    folium.GeoJson(
        row.geometry,  # The MultiPolygon geometry
        style_function=lambda x: {
            'fillColor': 'lawngreen',  # Color of the polygon
            'color': 'lawngreen',  # Border color of the polygon
            'weight': 1,  # Border thickness
            'fillOpacity': 0.3  # Fill opacity of the polygon
        },
        tooltip=folium.Tooltip(f"{round(row['potential_capacity']/1E3,2)} GW <br>"
                               f"Region: {row['Region']}<br>"
                               f"Rank: {row['Rank']}<br>"
                               f"LCOE ($/MWh): {round(row['LCOE'],2)}<br>"
                               )  # Show info on hover
    ).add_to(solar_clusters_group)

# Add points to the map without clustering
for _, row in filtered_rod_gdf.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),  # Lat, Long
        radius=row['Installed Capacity (MW)'] * 0.04,  # Scale marker size
        weight=.5,  # Thickness of the border
        color='orange' if row['Resource Type'] == 'Solar' else 'navy',  # Different colors for resource types
        fill=True,
        fill_opacity=0.3,
        popup=folium.Popup(f"<b>{row['Project Name']}</b><br>"
                           f"<b>{row['Resource Type']}</b><br>"
                           f"<strong>Rank: {int(row['Rank'])}</strong> <br>"
                           f"<strong>TUEC: {round(row['Total Adj UEC1 ($/MWh)'],2)}$/MWh</strong> <br>"
                           f"<strong>{int(row['Installed Capacity (MW)'])} MW</strong><br>"
                           f"<i>{row['Region']}</i>", max_width=250),  # Clean and readable popup
    ).add_to(points_group)

# Add all feature groups to the map
m.add_child(boundary_group)
m.add_child(wind_cells_group)
m.add_child(wind_clusters_group)
m.add_child(solar_clusters_group)
m.add_child(points_group)

# Add a sleek legend manually with modern style
legend_html = """
<div style="position: fixed; bottom: 30px; left: 30px; width: 250px; height: 160px; 
background-color: rgba(255, 255, 255, 0.7); border:2px solid #ccc; z-index:9999; font-size:14px; 
border-radius: 10px; padding: 10px;">
  <b>Legend</b><br>
  <i style="color:lawngreen;">●</i> Solar<br>
  <i style="color:navy;">●</i> Onshore Wind<br>
  <i style="color:violet;">●</i> Wind Clusters<br>
  <i style="color:lawngreen;">●</i> Solar Clusters<br>
  <i style="color:slategrey;">■</i> Boundary Region<br>  <!-- For boundary regions -->
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Add layer control to toggle visibility of the layers
folium.LayerControl().add_to(m)

# Save and display the map
m.save('ROD_2024_with_boundary.html')
m
